<a href="https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/interpolation_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Examples of comet_maths interpolation**

We first install and import our comet_maths package (and punpy, numpy and matplotlib), and define some example measurement functions:

In [ ]:
!pip install comet_maths

In [ ]:
import comet_maths as cm
import punpy

import numpy as np
import matplotlib.pyplot as plt

# your measurement function
def function1(x):
    return 15*x-20

def function2(x):
    """The function to predict."""
    return x * np.sin(x*10)

Next we define some example data, and do a simple interpolation without uncertainties, followed by various case with uncertainties.

In [ ]:
xi=np.arange(0,3.,0.2)
yi=function2(xi)
u_yi=0.05*yi

#add noise
yi = cm.generate_sample(1, yi, u_yi, corr_x="rand")

x=np.arange(0,2.5,0.01)

#It is possible to do interpolation without uncertainties
y=interpolate_1d(xi,yi,x,method="quadratic")

In [ ]:
#And with uncertainties (note in this case we don't specify any input uncertainties, so will only get model uncertainties)
y2,u_y2,corr_y2=interpolate_1d(xi,yi,x,method="cubic",add_error=False,return_uncertainties=True,return_corr=True)

In [ ]:
#This time with measurement uncertainties, this is more time consuming as needs to run MC
y3,u_y3,corr_y3=interpolate_1d(xi,yi,x,u_y_i=u_yi,method="cubic",add_error=False,return_uncertainties=True,return_corr=True)

In [ ]:
#While using gpr, random measurement uncertainties can also be propagated quickly (implemented inherently in the gpr algorithm)
y4,u_y4 = interpolate_1d(xi,yi,xx,method="gpr",u_y_i=u_yi, corr_y_i = "syst",min_scale=0.3,add_error=False,return_uncertainties=True)

In [ ]:
#However, when the uncertainties are not random (as defined in corr_y_i keyword), the propagation is slower as MC needs to be used 
y5,u_y5,corr_y5 = interpolate_1d(xi,yi,xx,method="gpr",u_y_i=u_yi, corr_y_i = "syst",min_scale=0.3,add_error=False,return_uncertainties=True,return_corr=True)

Next, we plot the results:

In [ ]:
fig=plt.figure(figsize=(10,5))
ax = fig.add_subplot(1, 1, 1)
ax.plot(x, function2(x), "k", label="true values")
ax.errorbar(xi, yi, yerr=u_yi, fmt="ko", ls=None, label="observed values")
ax.plot(x, y, "b:", label="quadratic interpolation")
ax.plot(x, y2, "r:", label="cubic interpolation (model error only)")
ax.fill_between(
x,
y2 - 1.9600 * u_y2,
(y2 + 1.9600 * u_y2),
alpha=0.25,
fc="r",
ec="None",
label="95% confidence interval",
lw=0,
)
ax.plot(x, y2, "m-", label="cubic interpolation")
ax.fill_between(
x,
y2 - 1.9600 * u_y2,
(y2 + 1.9600 * u_y2),
alpha=0.25,
fc="m",
ec="None",
label="95% confidence interval",
lw=0,
)
ax.plot(x, y4, "g", label="gpr interpolation")
ax.fill_between(
x,
y4 - 1.9600 * u_y4,
(y4 + 1.9600 * u_y4),
alpha=0.25,
fc="g",
ec="None",
label="95% confidence interval",
lw=0,
)
ax.plot(x, y5, "c", label="gpr interpolation with systematic measurement error")
ax.fill_between(
x,
y5 - 1.9600 * u_y5,
(y5 + 1.9600 * u_y5),
alpha=0.25,
fc="c",
ec="None",
label="95% confidence interval",
lw=0,
)
ax.legend()
fig.show()

fig2=plt.figure(figsize=(10,5))
ax = fig2.add_subplot(1, 3, 1)
ax2 = fig2.add_subplot(1, 3, 2)
ax3 = fig2.add_subplot(1, 3, 3)
p1=ax.imshow(corr_yy, vmin=-1, vmax=1, cmap="bwr")
ax.set_title("cubic interpolation (model error only)")
p2=ax2.imshow(corr_yy2, vmin=-1, vmax=1, cmap="bwr")
ax2.set_title("cubic interpolation")
p3=ax3.imshow(corr_yy2, vmin=-1, vmax=1, cmap="bwr")
ax3.set_title("gpr interpolation with systematic measurement error")
fig2.colorbar(p2)
fig2.show()

In [ ]:
        xi = np.arange(0, 3.0, 0.2)
        yi = function2(xi)
        u_yi = 0.01 * yi
        yi = cm.generate_sample(1, yi, u_yi, corr_x="rand").squeeze()
        x_HR = np.arange(0, 3.0, 0.036)
        y_HR = function2(x_HR)
        u_y_HR = 0.9 * np.ones_like(y_HR)
        corr_y_HR = np.ones((len(y_HR), len(y_HR)))  # + 0.05 * np.eye(len(y_HR))
        y_HR = cm.generate_sample(1, y_HR, u_y_HR, corr_x="syst")

        xx = np.arange(0, 2.5, 0.01)

        fig=plt.figure(figsize=(10,5))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(xx, function2(xx), "b", label="true values")
        ax.plot(xi, yi, "ro", label="LR observed values")
        ax.plot(x_HR, y_HR, "mo", label="HR observed values")
        ax.plot(xx, cm.interpolate_1d(xi, yi, xx, method="cubic"), "r:", label="cubic interpolation")
        ax.plot(xx, cm.interpolate_1d(x_HR, y_HR, xx, method="cubic"), "m:", label="cubic interpolation HR")
        ax.plot(
            xx,
            cm.interpolate_1d_along_example(
                xi, yi, x_HR, y_HR, xx, method="cubic", method_hr="cubic", relative=False
            ),
            "r-.", label="cubic interpolation along example"
        )
        ax.plot(
            xx,
            cm.interpolate_1d_along_example(
                xi, yi, x_HR, y_HR, xx, method="gpr", method_hr="gpr", relative=False
            ),
            "g-.", label="gpr interpolation along example"
        )
        ax.legend()
        fig.savefig("interpolation_test_1d_along_example.png", bbox_inches="tight")

    def test_interpolation_1d_along_example_unc(self):
        np.random.seed(123456)

        xi = np.arange(0, 3.0, 0.25)
        yi = function2(xi)
        u_yi = 0.03 * np.ones_like(yi)
        yi = cm.generate_sample(1, yi, u_yi, corr_x="rand").squeeze()
        x_HR = np.arange(0, 3.0, 0.09)
        y_HR = function2(x_HR)
        u_y_HR_syst = 0.9 * np.ones_like(y_HR)
        u_y_HR_rand = 0.02 * y_HR
        cov_y_HR = cm.convert_corr_to_cov(
            np.ones((len(y_HR), len(y_HR))), u_y_HR_syst
        ) + cm.convert_corr_to_cov(np.eye(len(y_HR)), u_y_HR_rand)
        corr_y_HR = cm.correlation_from_covariance(cov_y_HR)
        u_y_HR = cm.uncertainty_from_covariance(cov_y_HR)

        y_HR = cm.generate_sample(1, y_HR, u_y_HR, corr_x=corr_y_HR)

        xx = np.arange(0.1, 2.5, 0.02)

        y_hr_cubic = cm.interpolate_1d_along_example(
            xi,
            yi,
            x_HR,
            y_HR,
            xx,
            relative=False,
            method="cubic",
            method_hr="cubic",
            min_scale=0.3,
        )

        y_hr_cubic2, u_y_hr_cubic2 = cm.interpolate_1d_along_example(
            xi,
            yi,
            x_HR,
            y_HR,
            xx,
            relative=False,
            method="cubic",
            method_hr="cubic",
            u_y_i=u_yi,
            corr_y_i="rand",
            u_y_hr=u_y_HR,
            corr_y_hr="syst",
            min_scale=0.3,
            return_uncertainties=True,
            plot_residuals=True,
            return_corr=False,
        )

        npt.assert_allclose(y_hr_cubic,y_hr_cubic2,atol=0.01)

        y_gpr, u_y_gpr = cm.interpolate_1d(
            xi,
            yi,
            xx,
            method="gpr",
            u_y_i=u_yi,
            min_scale=0.3,
            return_uncertainties=True,
        )
        y_hr_gpr = cm.interpolate_1d_along_example(
            xi,
            yi,
            x_HR,
            y_HR,
            xx,
            relative=False,
            method="gpr",
            method_hr="gpr",
            min_scale=0.3,
        )
        y_hr_gpr2, u_y_hr_gpr2= cm.interpolate_1d_along_example(
            xi,
            yi,
            x_HR,
            y_HR,
            xx,
            relative=False,
            method="gpr",
            method_hr="gpr",
            u_y_i=u_yi,
            u_y_hr=u_y_HR,
            corr_y_i="rand",
            corr_y_hr=corr_y_HR,
            min_scale=0.3,
            return_uncertainties=True,
            plot_residuals=False,
            return_corr=False
        )

        # npt.assert_allclose(y_hr_gpr,y_hr_gpr2,atol=0.01)


        mcprop = punpy.MCPropagation(100, parallel_cores=4)

        inp2 = cm.Interpolator(
            relative=False, method="gpr", method_hr="gpr", min_scale=0.3,add_model_error=True
        )
        u_y_hr, corr2 = mcprop.propagate_random(
            inp2.interpolate_1d_along_example,
            [xi, yi, x_HR, y_HR, xx],
            [None, u_yi, None, u_y_HR, None],
            corr_x=[None, "rand", None, corr_y_HR, None],return_corr=True
        )

        #npt.assert_allclose(u_y_hr_cubic2, u_y_hr, rtol=0.01)
        fig=plt.figure(figsize=(10,5))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(xx, function2(xx), "b", label="True line")
        ax.plot(xi, yi, "ro", label="low-res data")
        ax.plot(x_HR, y_HR, "go", label="high-res data")
        ax.plot(
            xx,
            cm.interpolate_1d(xi, yi, xx, method="cubic"),
            "r:",
            label="cubic spline interpolation",
        )
        ax.plot(xx, y_gpr, "c:", label="GPR interpolation")
        ax.plot(xx, y_hr_gpr, "g-.", label="GPR interpolation with HR example")
        ax.plot(
            xx, y_hr_cubic, "m-.", label="cubic spline interpolation with HR example"
        )
        ax.fill_between(
            xx,
            y_hr_gpr2 - 1.9600 * u_y_hr,
            (y_hr_gpr2 + 1.9600 * u_y_hr),
            alpha=0.25,
            fc="c",
            ec="None",
            label="95% confidence interval",
            lw=0,
        )
        ax.fill_between(xx,y_hr_gpr2-1.9600*u_y_hr_gpr2,(y_hr_gpr2+1.9600*u_y_hr_gpr2),alpha=0.25,fc="g",ec="None",
                         label="95% confidence interval",lw=0)
        ax.fill_between(
            xx,
            y_hr_cubic2 - 1.9600 * u_y_hr_cubic2,
            (y_hr_cubic2 + 1.9600 * u_y_hr_cubic2),
            alpha=0.25,
            fc="m",
            ec="None",
            label="95% confidence interval",
            lw=0,
        )
        ax.legend(ncol=2, prop={"size": 6})
        ax.set_xlim([0.1, 2.5])
        # plt.show()
        fig.savefig("interpolation_test.pdf", bbox_inches="tight")

    def test_gaussian_process_regression(self):
        np.random.seed(1234567)

        xi = np.arange(0, 3.0, 0.25)
        yi = function2(xi)
        u_yi = 0.03 * np.ones_like(yi)
        yi = cm.generate_sample(1, yi, u_yi, corr_x="rand").squeeze()
        x_HR = np.arange(0, 3.0, 0.09)
        y_HR = function2(x_HR)
        u_y_HR_syst = 0.9 * np.ones_like(y_HR)
        u_y_HR_rand = 0.02 * y_HR
        cov_y_HR = cm.convert_corr_to_cov(
            np.ones((len(y_HR), len(y_HR))), u_y_HR_syst
        ) + cm.convert_corr_to_cov(np.eye(len(y_HR)), u_y_HR_rand)
        corr_y_HR = cm.correlation_from_covariance(cov_y_HR)
        u_y_HR = cm.uncertainty_from_covariance(cov_y_HR)

        y_HR = cm.generate_sample(1, y_HR, u_y_HR, corr_x=corr_y_HR)

        xx = np.arange(0.1, 2.5, 0.02)
        y_gpr,u_y_gpr = cm.gaussian_process_regression(
            xi,
            yi,
            xx,
            min_scale=0.3,
            return_uncertainties=True
        )

        y_gpr2,u_y_gpr2 = cm.gaussian_process_regression(
            xi,
            yi,
            xx,
            u_y_i=np.arange(len(u_yi))*u_yi,
            min_scale=0.3,
            return_uncertainties=True
        )

        y_gpr3,u_y_gpr3 = cm.gaussian_process_regression(
            xi,
            yi,
            xx,
            u_y_i=np.arange(len(u_yi))*u_yi,
            min_scale=0.3,
            return_uncertainties=True,
            include_model_uncertainties=False
        )

        u_y_gpr=(u_y_gpr**2+(u_y_gpr3)**2)**0.5


        y_hr_gpr,u_y_hr_gpr = cm.gaussian_process_regression(
            x_HR,
            y_HR,
            xx,
            u_y_i=u_y_HR,
            corr_y_i=corr_y_HR,
            min_scale=0.3,
            return_uncertainties=True
        )

        print(u_y_hr_gpr,u_y_HR)

        fig=plt.figure(figsize=(10,5))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(xx, function2(xx), "b", label="True line")
        ax.plot(xi, yi, "ro", label="low-res data")
        ax.plot(x_HR, y_HR, "go", label="high-res data")
        ax.plot(xx, y_hr_gpr, "g-.", label="GPR interpolation HR example")
        ax.fill_between(xx,y_hr_gpr-1.9600*u_y_hr_gpr,(y_hr_gpr+1.9600*u_y_hr_gpr),alpha=0.25,fc="g",ec="None",
                        label="95% confidence interval",lw=0)
        ax.plot(xx, y_gpr, "r-", label="GPR interpolation example")
        ax.fill_between(xx,y_gpr-1.9600*u_y_gpr,(y_gpr+1.9600*u_y_gpr),alpha=0.25,fc="r",ec="None",
                        label="95% confidence interval uncertainties added separately",lw=0)
        ax.plot(xx, y_gpr2, "m-", label="GPR interpolation example with combined unc")
        ax.fill_between(xx,y_gpr2-1.9600*u_y_gpr2,(y_gpr2+1.9600*u_y_gpr2),alpha=0.25,fc="m",ec="None",
                        label="95% confidence interval",lw=0)
        ax.legend(ncol=2, prop={"size": 6})
        ax.set_xlim([0.1, 2.5])
        fig.savefig("interpolation_gpr.png")

if __name__ == "__main__":
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        unittest.main()
